In [1]:
# Integration of biopsies and organ donors from 2 studies
import anndata, numpy as np, pandas as pd, scanpy as sc, imp, re, scvi
sc.logging.print_versions()
import lpy # local lpy.py file

Global seed set to 0
OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 9.0.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
deprecate           0.3.0
docrep              0.3.2
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5   

In [2]:
# Set some constants
adata = anndata.read_h5ad("int.h5ad")

In [3]:
# prepare data for scvi, subset out doublets, extrude cc genes (listed in function) and populate scvi objects 
flt = np.array((adata.obs["filtered_cells"] == False)&(adata.obs["Fraction"] != "CD3N"))
gflt = np.array(adata.var["feature_types"] == "Gene Expression")
adatasub = anndata.AnnData(X=adata.raw.X[flt,:][:,gflt],var=adata.raw.var[gflt],obs=adata.obs[flt])
adatasub.var_names_make_unique()
sc.pp.highly_variable_genes(
    adatasub,
    n_top_genes=5000,
    subset=True,
    flavor="seurat_v3"
)
adatasub = scvi.data.setup_anndata(adatasub , batch_key='sample_names',copy=True)
scvi.data.view_anndata_setup(adatasub)

INFO     Using batches from adata.obs["sample_names"]                                        


Using batches from adata.obs["sample_names"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.X                                                             


...

Anndata setup with scvi-tools version 0.10.1.

              Data Summary               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃             Data             ┃ Count  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│            Cells             │ 310946 │
│             Vars             │  5000  │
│            Labels            │   1    │
│           Batches            │   55   │
│           Proteins           │   0    │
│ Extra Categorical Covariates │   0    │
│ Extra Continuous Covariates  │   0    │
└──────────────────────────────┴────────┘

                SCVI Data Registry                 
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     Data      ┃       scvi-tools Location       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       X       │             adata.X             │
│ batch_indices │    adata.obs['_scvi_batch']     │
│ local_l_mean  │ adata.obs['_scvi_local_l_mean'] │
│  local_l_var  │ adata.obs['_scvi_local_l_var']  │
│    labels     │    adata.obs['_scvi_labels']    │
└───────────────┴─────────────────────────────────┘

                        Label Categories                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                         Batch Categories                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃  Categories   ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['sample_names'] │ BGCV01_CV0902 │          0          │
│                           │ BGCV01_CV0904 │          1          │
│                           │ BGCV02_CV0902 │          2          │
│                           │ BGCV04_CV0911 │          3          │
│                           │ BGCV05_CV0929 │          4          │
│                           │ BGCV08_CV0915 │          5          │
│                           │ BGCV09_CV0917 │          6          │
│                           │ BGCV10_CV0939 │          7          │
│                           │ BGCV12_CV0926 │          8          │
│                           │ BGCV13_CV0934 │          9          │
│                           │ BGCV14_CV0940 │         10          │
│                           │ BGCV15_CV0944 │         11          │
│                           │  I-O-5_8_C-0  │         12          │
│                           │  I-O-6_9_C-0  │         13          │
│                           │   I-O-7_C-0   │         14          │
│                           │   MH8919176   │         15          │
│                           │   MH8919177   │         16          │
│                           │   MH8919178   │         17          │
│                           │   MH8919179   │         18          │
│                           │   MH8919226   │         19          │
│                           │   MH8919227   │         20          │
│                           │   MH8919282   │         21          │
│                           │   MH8919283   │         22          │
│                           │   MH8919332   │         23          │
│                           │   MH8919333   │         24          │
│                           │    S-HC008    │         25          │
│                           │    S-HC010    │         26          │
│                           │    S-HC013    │         27          │
│                           │    S-HC014    │         28          │
│                           │    S-HC015    │         29          │
│                           │    S-HC017    │         30          │
│                           │   S-M044-1    │         31          │
│                           │   S-M044-2    │         32          │
│                           │    S-M046     │         33          │
│                           │    S-M047     │         34          │
│                           │    S-M050     │         35          │
│                           │    S-M051     │         36          │
│                           │    S-M052     │         37          │
│                           │    S-M055     │         38          │
│                           │    S-M067     │         39          │
│                           │    S-S054     │         40          │
│                           │    S-S055     │         41          │
│                           │    S-S057     │         42          │
│                           │    S-S059     │         43          │
│                           │    S-S061     │         44          │
│                           │    S-S062     │         45          │
│                           │    S-S064     │         46          │
│                           │    S-S065     │         47          │
│                           │    S-S067     │         48          │
│                           │    S-S068     │         49          │
│                           │    T1_C-0     │         50          │
│                           │    T2_C-0     │         51          │
│                           │    T3_C-0     │         52          │
│                           │  newcastle65  │         53          │
│                           │  newcastle74  │         54

In [4]:
scvimodel = scvi.model.SCVI(adatasub)
scvimodel.train()
latent = scvimodel.get_latent_representation()

adata = lpy.insertLatent(adata,latent, "scvi_sampl_everything", "X_umap_sampl_everything", None, "leiden_sampl_everything", cellnames = adatasub.obs_names, doinspect=True)
adata = lpy.insertLatent(adata,latent, None, None, None, "leidenres2_sampl_everything", cellnames = adatasub.obs_names,leiden_resolution=2)
adata.write_h5ad("int2.h5ad")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 26/26: 100%|██████████| 26/26 [3:00:20<00:00, 416.19s/it, loss=928, v_num=1]  
Compute Clusters and Reduces representations
def insertLatent(adata, latent , latent_key= "latent", umap_key= "X_umap", tsne_key = "X_tsne", leiden_key = "leiden", rename_cluster_key= None,cellfilter = None, cellnames =None, leiden_resolution=1.0,doinspect=False):
    if doinspect is True: print("\033Compute Clusters and Reduces representations\033\033"); print(inspect.getsource(insertLatent));print("\033Execution:\033")
    import umap
    #from umap import UMAP
    #from umap.umap_ import UMAP
    if cellnames is None:
        #order of full must match
        assert latent.shape[0] == len(adata.obs_names),  "cell names need for be provided if size of latent mismatches adata"
        map = range(len(adata.obs_names))
        if latent_key is not None:
            adata.obsm[latent_key] = latent
    else:
        print("defining permutation")
        dalist = list(adata.obs_names)
        tmap = {}
  

... storing 'leiden_sampl_everything' as categorical
... storing 'leidenres2_sampl_everything' as categorical
